In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from pprint import pprint as pp
import time


In [4]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import platform
path = None
if platform.system() is 'Windows':
    path = './drivers/geckodriver.exe'
else:
    path = './drivers/geckodriver'
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options,executable_path=path)

In [7]:
url_data = pd.read_csv("./dataset/url.csv")
url_data.head()

,url,date
0,https://www.fox8live.com/2019/09/07/police-cou...,9/7/19
1,https://q13fox.com/2019/11/13/bellevue-police-...,11/13/2019
2,https://www.taiwannews.com.tw/en/news/3819595,11/17/2019
3,https://twitter.com/Woppa1Woppa/status/1196315...,11/17/2019


In [8]:
def html_decode(s):
    """
    Returns the ASCII decoded version of the given HTML string. This does
    NOT remove normal HTML tags like <p>.
    """
    htmlCodes = (
            ("'", '&#39;'),
            ('"', '&quot;'),
            ('>', '&gt;'),
            ('<', '&lt;'),
            ('&', '&amp;')
        )
    for code in htmlCodes:
        s = s.replace(code[1], code[0])
    return s

In [9]:
def get_page_source(url):
    driver.get(url)
    return driver.page_source

In [10]:
scrapping_url = "https://embed.ly/docs/explore/extract?url="
apikey = "1934f519a63e142e0d3c893e59cc37fe0172e98a"
sharedcount_url = "https://api.sharedcount.com/v1.0/?url={}&apikey={}"
reddit_search = "https://www.reddit.com/domain/{}/search?q={}&sort=top"
colon = '%3A'
fw_slash = '%2F'
def get_sharedcount_url(url):
    return sharedcount_url.format(url,apikey)

def get_fbengagement(url):
    shared_count = json.loads(requests.get(get_sharedcount_url(url)).text)
    fb = shared_count['Facebook']
    return fb

def scrape_content(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    json_text = html_decode(soup.pre.text)
    row = json.loads(json_text)
    if row['content'] is None and row['description'] is None:
        print("Skipping link : ",url)
        return None
    if row['content'] is None and row['description'] is not None :
        row['content'] = row['description']
    row['content']= BeautifulSoup(row['content'], "lxml").text
    row['provider_display'] = row['provider_display'].replace('www.','')
    return row

def scrape_reddit(domain, title):
    query_url = reddit_search.format(domain,title.replace(" ","+"))
#     page = requests.get(query_url).text
    page = get_page_source(query_url)
    soup = BeautifulSoup(page,'html.parser')
    upvote = list()
    for score in soup.findAll('span',{'class':'search-score'}):
        upvote += [int(i) for i in score.text.split() if i.isdigit()] 
    comments = list()
    for score in soup.findAll('a',{'class':'search-comments'}):
        comments += [int(i) for i in score.text.split() if i.isdigit()] 
    return (sum(upvote),sum(comments))

def scrape_news(data):
    for url in data:
        url = url.replace(':',colon)
        url = url.replace('/',fw_slash)
        article = scrape_content(scrapping_url+url)
        if article is None:
            continue
        article.update(get_fbengagement(url))
        article['reddit_upvotes'],article['reddit_comments'] = scrape_reddit(article['provider_display'],article['title'])
        yield article

for row in scrape_news(url_data['url']):
    print(row['title'])
    


Police: Couple charged after spending more than $100K the bank accidentally gave them
Bellevue Police arrest 12 men in prostitution sting
Video sparks fears Hong Kong protesters being... | Taiwan News
woppa 🎗😷 on Twitter


In [11]:
def create_data(data):
    dataset = list()
    i = 0
    for row in scrape_news(data):
        dataset+=[row]
    return dataset

In [12]:

df = pd.DataFrame(create_data(url_data['url']))
df = df[['language','title','content','description','provider_url','keywords','provider_display','provider_name',
            'total_count','comment_count','share_count','reaction_count','reddit_upvotes','reddit_comments']]
df['date'] = url_data['date']
df['date'] = pd.to_datetime(df['date'])
df['title_len'] = df['title'].str.split().apply(len)
df['content_len'] = df['content'].str.split().apply(len)
df['description_len'] = df['description'].str.split().apply(len)

df.head()

,language,title,content,description,provider_url,keywords,provider_display,provider_name,total_count,comment_count,share_count,reaction_count,reddit_upvotes,reddit_comments,date,title_len,content_len,description_len
0,en,Police: Couple charged after spending more tha...,"\n\n\nMONTOURSVILLE, Pa. (WNEP/CNN) - A couple...","MONTOURSVILLE, Pa. (WNEP/CNN) - A couple in Pe...",https://www.fox8live.com,"[{'score': 0, 'name': 'bank'}, {'score': 0, 'n...",fox8live.com,https://www.fox8live.com,998,399,178,421,8,24,2019-09-07,13,283,54
1,en,Bellevue Police arrest 12 men in prostitution ...,"\nBELLEVUE, Wash. - Bellevue Police Department...","BELLEVUE, Wash. - Bellevue Police Department d...",https://q13fox.com,"[{'score': 0, 'name': 'crime'}, {'score': 0, '...",q13fox.com,Q13 FOX News,309,38,57,214,11,4,2019-11-13,8,384,52
2,None,Video sparks fears Hong Kong protesters being....,\n TAIPEI (Taiwan News) - A video that surface...,Video of Hong Kong protesters being loaded ont...,https://www.taiwannews.com.tw,[],taiwannews.com.tw,Taiwan News,1624,258,880,486,927,592,2019-11-17,10,490,54
3,en,woppa 🎗😷 on Twitter,Arrested protesters are getting transported ou...,Arrested protesters are getting transported ou...,http://twitter.com,[],twitter.com,Twitter,124,16,35,73,0,0,2019-11-17,4,32,32


In [13]:
df.to_csv("./dataset/meta.csv")

In [14]:
driver.quit()